In [1]:
# Importando bibliotecas
import pandas as pd
from datetime import datetime

# Lendo o arquivo csv e alocando-o em um dataset
dados = pd.read_csv('campeonato-brasileiro-full.csv')

In [2]:
# Exibindo o dataset
display(dados)

,ID,rodata,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado
0,1,1,29/03/2003,16:00,Guarani,Vasco,NaN,NaN,NaN,NaN,Guarani,Brinco de Ouro,4,2,SP,RJ
1,2,1,29/03/2003,16:00,Athletico-PR,Gremio,NaN,NaN,NaN,NaN,Athletico-PR,Arena da Baixada,2,0,PR,RS
2,3,1,30/03/2003,16:00,Flamengo,Coritiba,NaN,NaN,NaN,NaN,-,Maracanã,1,1,RJ,PR
3,4,1,30/03/2003,16:00,Goias,Paysandu,NaN,NaN,NaN,NaN,-,Serra Dourada,2,2,GO,PA
4,5,1,30/03/2003,16:00,Internacional,Ponte Preta,NaN,NaN,NaN,NaN,-,Beira Rio,1,1,RS,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8400,8401,38,06/12/2023,21:32,Bahia,Atletico-MG,3-4-2-1,4-4-2,R. Ceni,L. Scolari,Bahia,Itaipava Arena Fonte Nova,4,1,BA,MG
8401,8402,38,06/12/2023,21:32,Cuiaba,Athletico-PR,5-3-2,3-4-3,A. Cardoso de Oliveira,W. Carvalho,Cuiaba,Arena Pantanal,3,0,MT,PR
8402,8403,38,06/12/2023,21:32,Santos,Fortaleza,3-4-1-2,4-2-3-1,M. Fernandes,J. Vojvoda,Fortaleza,Estádio Urbano Caldeira,1,2,SP,CE
8403,8404,38,06/12/2023,21:32,Sao Paulo,Flamengo,4-2-3-1,4-2-3-1,D. Silvestre Júnior,A. Bacchi,Sao Paulo,Morumbi,1,0,SP,RJ


**Editando o dataframe**

In [3]:
# Criando uma cópia do dataframe
dados_processado = dados.copy()

In [4]:
# Percebemos que a coluna rodata está escrita de forma errada. Vamos renomeá-la para rodada
dados_processado.rename(columns={'rodata': 'rodada'}, inplace=True)

# Da mesma forma, vamos excluir a coluna ID pois não será necessária, visto que já temos os índices do dataframe
del dados_processado['ID']

In [5]:
dados_processado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8405 entries, 0 to 8404
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   rodada              8405 non-null   int64 
 1   data                8405 non-null   object
 2   hora                8405 non-null   object
 3   mandante            8405 non-null   object
 4   visitante           8405 non-null   object
 5   formacao_mandante   3431 non-null   object
 6   formacao_visitante  3431 non-null   object
 7   tecnico_mandante    3795 non-null   object
 8   tecnico_visitante   3795 non-null   object
 9   vencedor            8405 non-null   object
 10  arena               8405 non-null   object
 11  mandante_Placar     8405 non-null   int64 
 12  visitante_Placar    8405 non-null   int64 
 13  mandante_Estado     8405 non-null   object
 14  visitante_Estado    8405 non-null   object
dtypes: int64(3), object(12)
memory usage: 985.1+ KB


In [6]:
# Convertendo coluna data para datetime
dados_processado['data'] = pd.to_datetime(dados['data'], dayfirst=True) 

In [7]:
# Criando nova coluna no dataframe para armazenar o dia da semana em que a partida foi realizada
dados_processado.insert(3, 'dia', dados_processado['data'].dt.dayofweek)

# Mapeando os números para os nomes dos dias da semana
dados_processado['dia'] = dados_processado['dia'].map({0: 'Segunda-feira', 1: 'Terça-feira', 2: 'Quarta-feira', 3: 'Quinta-feira', 4: 'Sexta-feira', 5: 'Sábado', 6: 'Domingo'})

In [8]:
# Alterando para 'Empate' os valores '-' na coluna vencedor
dados_processado['vencedor'] = dados_processado['vencedor'].replace('-', 'Empate')

In [9]:
# Definindo função que retorna o perdedor da partida
def retornar_perdedor(mandante, visitante, vencedor):
    if vencedor == mandante:
        return visitante
    elif vencedor == visitante:
        return mandante
    else:
        return 'Empate'
    
perdedores = []
# Criando coluna perdedor
for linha in range(dados_processado.shape[0]):
    perdedores.append(retornar_perdedor(dados_processado['mandante'][linha], dados_processado['visitante'][linha], dados_processado['vencedor'][linha]))
    
dados_processado.insert(11, 'perdedor', perdedores)

In [10]:
# Visualizando como ficou o dataset editado
dados_processado.head()

,rodada,data,hora,dia,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,perdedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado
0,1,2003-03-29,16:00,Sábado,Guarani,Vasco,NaN,NaN,NaN,NaN,Guarani,Vasco,Brinco de Ouro,4,2,SP,RJ
1,1,2003-03-29,16:00,Sábado,Athletico-PR,Gremio,NaN,NaN,NaN,NaN,Athletico-PR,Gremio,Arena da Baixada,2,0,PR,RS
2,1,2003-03-30,16:00,Domingo,Flamengo,Coritiba,NaN,NaN,NaN,NaN,Empate,Empate,Maracanã,1,1,RJ,PR
3,1,2003-03-30,16:00,Domingo,Goias,Paysandu,NaN,NaN,NaN,NaN,Empate,Empate,Serra Dourada,2,2,GO,PA
4,1,2003-03-30,16:00,Domingo,Internacional,Ponte Preta,NaN,NaN,NaN,NaN,Empate,Empate,Beira Rio,1,1,RS,SP


In [11]:
# Salvando o dataset processado na variável dados
dados = dados_processado

**Editando dados para utilizar na análise**

In [12]:
# Criando array que contém todos os times que disputaram o Brasileirão por pontos corridos
times = dados['mandante'].unique()
times.sort()
times = list(times)

# Criando dataframe que guarde os dados dos times
dados_times = pd.DataFrame()
dados_times['time'] = times
dados_times.head()

,time
0,America-MG
1,America-RN
2,Athletico-PR
3,Atletico-GO
4,Atletico-MG


In [13]:
# Criando dataframes para guardar os dados dos times como mandante e como visitante
dados_mandantes = pd.DataFrame()
dados_mandantes['time'] = dados_times.copy()

dados_visitantes = pd.DataFrame()
dados_visitantes['time'] = dados_times.copy()


In [14]:
# Inserindo a quantidade de jogos de cada time ao dataframe
contagem_jogos = dados['mandante'].value_counts() + dados['visitante'].value_counts()
dados_times.insert(1, 'jogos', contagem_jogos[dados_times['time']].values)

dados_times.head()

,time,jogos
0,America-MG,228
1,America-RN,38
2,Athletico-PR,780
3,Atletico-GO,266
4,Atletico-MG,779


In [15]:
# Criando dataframe que contém todos os jogos sem os empates
selecao = dados['vencedor'] != 'Empate'
dados_sem_empate = dados[selecao]

In [16]:
# Adicionando a quantidade de vitórias de cada time ao dataframe
vitorias = dados_sem_empate['vencedor'].value_counts()
vitorias = vitorias.sort_index()
dados_times.insert(2, 'vitórias', vitorias.values)

dados_times.head()

,time,jogos,vitórias
0,America-MG,228,58
1,America-RN,38,4
2,Athletico-PR,780,309
3,Atletico-GO,266,72
4,Atletico-MG,779,327


In [17]:
# Adicionando as derrotas de cada time
derrotas = dados_sem_empate['perdedor'].value_counts()
derrotas = derrotas.sort_index()
dados_times.insert(3, 'derrotas', derrotas.values)

dados_times.head()

,time,jogos,vitórias,derrotas
0,America-MG,228,58,109
1,America-RN,38,4,29
2,Athletico-PR,780,309,282
3,Atletico-GO,266,72,115
4,Atletico-MG,779,327,253


In [18]:
# Adicionando as vitórias como mandante de cada time
vitorias_casa = dados_sem_empate.query('vencedor == mandante')
vitorias = vitorias_casa['vencedor'].value_counts()
vitorias = vitorias.sort_index()
dados_mandantes.insert(1, 'vitórias em casa', vitorias.values)

dados_mandantes.head()

,time,vitórias em casa
0,America-MG,45
1,America-RN,2
2,Athletico-PR,221
3,Atletico-GO,43
4,Atletico-MG,218


In [19]:
# Adicionando as vitórias como visitante de cada time
vitorias_fora = dados_sem_empate.query('vencedor == visitante')
vitorias = vitorias_fora['vencedor'].value_counts()

# Após erro, foi constatado que um time não possui vitória como visitante, sendo este time o Ipatinga
# Adicionando Ipatinga
vitorias['Ipatinga'] = 0
vitorias = vitorias.sort_index()

dados_visitantes.insert(1, 'vitórias fora de casa', vitorias.values)

dados_visitantes.head()

,time,vitórias fora de casa
0,America-MG,13
1,America-RN,2
2,Athletico-PR,88
3,Atletico-GO,29
4,Atletico-MG,109


In [20]:
# Adicionando as derrotas como mandante de cada time
derrotas_casa = dados_sem_empate.query('perdedor == mandante')
derrotas = derrotas_casa['perdedor'].value_counts()
derrotas = derrotas.sort_index()
dados_mandantes.insert(2, 'derrotas em casa', derrotas.values)

dados_mandantes.head()

,time,vitórias em casa,derrotas em casa
0,America-MG,45,40
1,America-RN,2,14
2,Athletico-PR,221,67
3,Atletico-GO,43,49
4,Atletico-MG,218,89


In [21]:
# Adicionando as derrotas como visitante de cada time
derrotas_fora = dados_sem_empate.query('perdedor == visitante')
derrotas = derrotas_fora['perdedor'].value_counts()
derrotas = derrotas.sort_index()
dados_visitantes.insert(2, 'derrotas fora de casa', derrotas.values)

dados_visitantes.head()

,time,vitórias fora de casa,derrotas fora de casa
0,America-MG,13,69
1,America-RN,2,15
2,Athletico-PR,88,215
3,Atletico-GO,29,66
4,Atletico-MG,109,164


In [22]:
# Criando dataframe apenas com os jogos que terminaram empatados
selecao = dados['vencedor'] == 'Empate'
dados_empates = dados[selecao]

In [23]:
# Adicionando os empates em casa ao dataframe
empates = dados_empates['mandante'].value_counts()
empates = empates.sort_index()
dados_mandantes.insert(2, 'empates em casa', empates.values)

dados_mandantes.head()

,time,vitórias em casa,empates em casa,derrotas em casa
0,America-MG,45,29,40
1,America-RN,2,3,14
2,Athletico-PR,221,102,67
3,Atletico-GO,43,41,49
4,Atletico-MG,218,83,89


In [24]:
# Adicionando os empates fora de casa ao dataframe
empates = dados_empates['visitante'].value_counts()
empates = empates.sort_index()
dados_visitantes.insert(2, 'empates fora de casa', empates.values)

dados_visitantes.head()

,time,vitórias fora de casa,empates fora de casa,derrotas fora de casa
0,America-MG,13,32,69
1,America-RN,2,2,15
2,Athletico-PR,88,87,215
3,Atletico-GO,29,38,66
4,Atletico-MG,109,116,164


In [25]:
# Adicionando os empates totais ao dataframe
dados_times.insert(3, 'empates', dados_mandantes['empates em casa'] + dados_visitantes['empates fora de casa'])
                   
dados_times.head()

,time,jogos,vitórias,empates,derrotas
0,America-MG,228,58,61,109
1,America-RN,38,4,5,29
2,Athletico-PR,780,309,189,282
3,Atletico-GO,266,72,79,115
4,Atletico-MG,779,327,199,253


In [26]:
# Adicionando ao dataframe a quantidade de jogos em casa e fora de casa
contagem_jogos = dados['mandante'].value_counts()
dados_mandantes.insert(1, 'jogos em casa', contagem_jogos[dados_mandantes['time']].values)

contagem_jogos = dados['visitante'].value_counts()
dados_visitantes.insert(1, 'jogos fora de casa', contagem_jogos[dados_visitantes['time']].values)

In [27]:
dados_mandantes.head()

,time,jogos em casa,vitórias em casa,empates em casa,derrotas em casa
0,America-MG,114,45,29,40
1,America-RN,19,2,3,14
2,Athletico-PR,390,221,102,67
3,Atletico-GO,133,43,41,49
4,Atletico-MG,390,218,83,89


In [28]:
dados_visitantes.head()

,time,jogos fora de casa,vitórias fora de casa,empates fora de casa,derrotas fora de casa
0,America-MG,114,13,32,69
1,America-RN,19,2,2,15
2,Athletico-PR,390,88,87,215
3,Atletico-GO,133,29,38,66
4,Atletico-MG,389,109,116,164


In [29]:
# Definindo função para realizar contagem de pontos
def contar_pontos(num_vitorias, num_empates):
    return num_vitorias * 3 + num_empates * 1

# Adicionando total de pontos, pontos em casa e pontos fora de casa ao dataframe
dados_times.insert(2, 'pontos', contar_pontos(dados_times['vitórias'], dados_times['empates']))
dados_mandantes.insert(2, 'pontos em casa', contar_pontos(dados_mandantes['vitórias em casa'], dados_mandantes['empates em casa']))
dados_visitantes.insert(2, 'pontos fora de casa', contar_pontos(dados_visitantes['vitórias fora de casa'], dados_visitantes['empates fora de casa']))

dados_times.head()

,time,jogos,pontos,vitórias,empates,derrotas
0,America-MG,228,235,58,61,109
1,America-RN,38,17,4,5,29
2,Athletico-PR,780,1116,309,189,282
3,Atletico-GO,266,295,72,79,115
4,Atletico-MG,779,1180,327,199,253


In [30]:
dados_mandantes.head()

,time,jogos em casa,pontos em casa,vitórias em casa,empates em casa,derrotas em casa
0,America-MG,114,164,45,29,40
1,America-RN,19,9,2,3,14
2,Athletico-PR,390,765,221,102,67
3,Atletico-GO,133,170,43,41,49
4,Atletico-MG,390,737,218,83,89


In [31]:
dados_visitantes.head()

,time,jogos fora de casa,pontos fora de casa,vitórias fora de casa,empates fora de casa,derrotas fora de casa
0,America-MG,114,71,13,32,69
1,America-RN,19,8,2,2,15
2,Athletico-PR,390,351,88,87,215
3,Atletico-GO,133,125,29,38,66
4,Atletico-MG,389,443,109,116,164


In [32]:
# Definindo função para verificar aproveitamento dos times
def retornar_aproveitamento(num_jogos, pontos):
    pontos_em_disputa = num_jogos * 3
    return round(100 / pontos_em_disputa * pontos, 2) 

# Inserindo aproveitamento total, como mandante e como visitante dos times
dados_times.insert(6, 'aproveitamento', retornar_aproveitamento(dados_times['jogos'], dados_times['pontos']))
dados_mandantes.insert(6, 'aproveitamento em casa', retornar_aproveitamento(dados_mandantes['jogos em casa'], dados_mandantes['pontos em casa']))
dados_visitantes.insert(6, 'aproveitamento fora de casa', retornar_aproveitamento(dados_visitantes['jogos fora de casa'], dados_visitantes['pontos fora de casa']))

dados_times.head()

,time,jogos,pontos,vitórias,empates,derrotas,aproveitamento
0,America-MG,228,235,58,61,109,34.36
1,America-RN,38,17,4,5,29,14.91
2,Athletico-PR,780,1116,309,189,282,47.69
3,Atletico-GO,266,295,72,79,115,36.97
4,Atletico-MG,779,1180,327,199,253,50.49


In [33]:
dados_mandantes.head()

,time,jogos em casa,pontos em casa,vitórias em casa,empates em casa,derrotas em casa,aproveitamento em casa
0,America-MG,114,164,45,29,40,47.95
1,America-RN,19,9,2,3,14,15.79
2,Athletico-PR,390,765,221,102,67,65.38
3,Atletico-GO,133,170,43,41,49,42.61
4,Atletico-MG,390,737,218,83,89,62.99


In [34]:
dados_visitantes.head()

,time,jogos fora de casa,pontos fora de casa,vitórias fora de casa,empates fora de casa,derrotas fora de casa,aproveitamento fora de casa
0,America-MG,114,71,13,32,69,20.76
1,America-RN,19,8,2,2,15,14.04
2,Athletico-PR,390,351,88,87,215,30.00
3,Atletico-GO,133,125,29,38,66,31.33
4,Atletico-MG,389,443,109,116,164,37.96


In [35]:
# Realizando contagem de gols
gols_mandante = dados[['mandante','mandante_Placar']]
gols_mandante = gols_mandante.groupby('mandante').sum().sort_index()

gols_visitante = dados[['visitante','visitante_Placar']]
gols_visitante = gols_visitante.groupby('visitante').sum().sort_index()

# Inserindo aos dataframes os gols em casa e gols fora
dados_mandantes.insert(6, 'gols pró em casa', gols_mandante.values)
dados_visitantes.insert(6, 'gols pró fora de casa', gols_visitante.values)

In [36]:
# Inserindo ao dataframe os gols totais
dados_times.insert(6, 'gols pró', gols_mandante.values + gols_visitante.values)
dados_times.head()

,time,jogos,pontos,vitórias,empates,derrotas,gols pró,aproveitamento
0,America-MG,228,235,58,61,109,227,34.36
1,America-RN,38,17,4,5,29,24,14.91
2,Athletico-PR,780,1116,309,189,282,1032,47.69
3,Atletico-GO,266,295,72,79,115,288,36.97
4,Atletico-MG,779,1180,327,199,253,1131,50.49


In [37]:
# Realizando contagem de gols sofridos
gols_sofridos_mandante = dados[['mandante','visitante_Placar']]
gols_sofridos_mandante = gols_sofridos_mandante.groupby('mandante').sum().sort_index()

gols_sofridos_visitante = dados[['visitante','mandante_Placar']]
gols_sofridos_visitante = gols_sofridos_visitante.groupby('visitante').sum().sort_index()

# Inserindo aos dataframes os gols em casa e gols fora
dados_mandantes.insert(7, 'gols sofridos em casa', gols_sofridos_mandante.values)
dados_visitantes.insert(7, 'gols sofridos fora de casa', gols_sofridos_visitante.values)

In [38]:
# Inserindo ao dataframe os gols sofridos totais
dados_times.insert(7, 'gols sofridos', gols_sofridos_mandante.values + gols_sofridos_visitante.values)
dados_times.head()

,time,jogos,pontos,vitórias,empates,derrotas,gols pró,gols sofridos,aproveitamento
0,America-MG,228,235,58,61,109,227,332,34.36
1,America-RN,38,17,4,5,29,24,80,14.91
2,Athletico-PR,780,1116,309,189,282,1032,965,47.69
3,Atletico-GO,266,295,72,79,115,288,363,36.97
4,Atletico-MG,779,1180,327,199,253,1131,978,50.49


In [39]:
# Agrupando dataframes
dados_times_full = pd.merge(dados_times, dados_mandantes, on='time')
dados_times_full = pd.merge(dados_times_full, dados_visitantes, on='time')

dados_times_full.head()

,time,jogos,pontos,vitórias,empates,derrotas,gols pró,gols sofridos,aproveitamento,jogos em casa,...,gols sofridos em casa,aproveitamento em casa,jogos fora de casa,pontos fora de casa,vitórias fora de casa,empates fora de casa,derrotas fora de casa,gols pró fora de casa,gols sofridos fora de casa,aproveitamento fora de casa
0,America-MG,228,235,58,61,109,227,332,34.36,114,...,131,47.95,114,71,13,32,69,94,201,20.76
1,America-RN,38,17,4,5,29,24,80,14.91,19,...,35,15.79,19,8,2,2,15,12,45,14.04
2,Athletico-PR,780,1116,309,189,282,1032,965,47.69,390,...,357,65.38,390,351,88,87,215,388,608,30.00
3,Atletico-GO,266,295,72,79,115,288,363,36.97,133,...,149,42.61,133,125,29,38,66,130,214,31.33
4,Atletico-MG,779,1180,327,199,253,1131,978,50.49,390,...,408,62.99,389,443,109,116,164,454,570,37.96


In [40]:
# Alterando indexes para nome do time
dados_times_full.set_index('time', inplace=True)

dados_times_full.head()

,jogos,pontos,vitórias,empates,derrotas,gols pró,gols sofridos,aproveitamento,jogos em casa,pontos em casa,...,gols sofridos em casa,aproveitamento em casa,jogos fora de casa,pontos fora de casa,vitórias fora de casa,empates fora de casa,derrotas fora de casa,gols pró fora de casa,gols sofridos fora de casa,aproveitamento fora de casa
time,,,,,,,,,,,,,,,,,,,,,
America-MG,228,235,58,61,109,227,332,34.36,114,164,...,131,47.95,114,71,13,32,69,94,201,20.76
America-RN,38,17,4,5,29,24,80,14.91,19,9,...,35,15.79,19,8,2,2,15,12,45,14.04
Athletico-PR,780,1116,309,189,282,1032,965,47.69,390,765,...,357,65.38,390,351,88,87,215,388,608,30.00
Atletico-GO,266,295,72,79,115,288,363,36.97,133,170,...,149,42.61,133,125,29,38,66,130,214,31.33
Atletico-MG,779,1180,327,199,253,1131,978,50.49,390,737,...,408,62.99,389,443,109,116,164,454,570,37.96


In [41]:
# Salvando dataframe
dados_times_full.to_csv('C:/Users/User/ciencia-dados/brasileirao/dados-times-brasileirao-2003-2023-full.csv')

**ANÁLISE**

In [42]:
# Selecionando times com menos jogos no campeonato (equipes que participaram apenas uma vez)
selecao = dados_times_full['jogos'] <= 42
dados_jogos = dados_times_full[selecao]

dados_jogos['jogos']

time
America-RN         38
Barueri            38
Brasiliense        42
CSA                38
Gremio Prudente    38
Ipatinga           38
Joinville          38
Santo Andre        38
Name: jogos, dtype: int64

In [54]:
# Selecionando times com mais jogos no campeonato 
dados_jogos = dados_times_full['jogos']
top_jogos = dados_jogos.sort_values(ascending=False)
top_jogos.head(10)

time
Santos           818
Flamengo         818
Sao Paulo        818
Fluminense       818
Athletico-PR     780
Corinthians      780
Internacional    780
Atletico-MG      779
Gremio           738
Palmeiras        734
Name: jogos, dtype: int64

In [55]:
# Selecionando times que mais venceram no campeonato
dados_vitorias = dados_times_full['vitórias']
top_vitorias = dados_vitorias.sort_values(ascending=False)
top_vitorias.head()

time
Sao Paulo        365
Flamengo         354
Santos           340
Internacional    339
Palmeiras        329
Name: vitórias, dtype: int64

In [56]:
# Selecionando times que mais perderam no campeonato
dados_derrotas = dados_times_full['derrotas']
top_derrotas = dados_derrotas.sort_values(ascending=False)
top_derrotas.head()

time
Fluminense      283
Athletico-PR    282
Santos          261
Botafogo-RJ     259
Atletico-MG     253
Name: derrotas, dtype: int64

In [57]:
# Selecionando times que mais empataram no campeonato
dados_empates = dados_times_full['empates']
top_empates = dados_empates.sort_values(ascending=False)
top_empates.head(10)

time
Sao Paulo        229
Corinthians      229
Flamengo         221
Santos           217
Fluminense       213
Internacional    202
Atletico-MG      199
Botafogo-RJ      190
Palmeiras        190
Athletico-PR     189
Name: empates, dtype: int64

In [58]:
# Selecionando os times com melhores aproveitamentos
dados_aproveitamento = dados_times_full['aproveitamento']
top_aproveitamentos = dados_aproveitamento.sort_values(ascending=False)
top_aproveitamentos.head(10)

time
Sao Paulo        53.95
Palmeiras        53.45
Flamengo         52.28
Internacional    52.09
Corinthians      51.58
Gremio           51.17
Cruzeiro         50.99
Atletico-MG      50.49
Santos           50.41
Fluminense       48.04
Name: aproveitamento, dtype: float64

In [59]:
# Selecionando os times com piores aproveitamentos
piores_aproveitamentos = dados_aproveitamento.sort_values()
piores_aproveitamentos.head(10)

time
America-RN         14.91
Santa Cruz         25.88
Gremio Prudente    27.19
Joinville          27.19
CSA                28.07
Ipatinga           30.70
Brasiliense        33.33
Avai               33.96
America-MG         34.36
Nautico            35.09
Name: aproveitamento, dtype: float64

In [64]:
# Selecionando times com melhores aproveitamentos como mandante
dados_aproveitamento_casa = dados_times_full['aproveitamento em casa']
top_aproveitamentos = dados_aproveitamento_casa.sort_values(ascending=False)
top_aproveitamentos

time
Gremio             66.76
Internacional      66.67
Sao Paulo          66.26
Athletico-PR       65.38
Palmeiras          64.94
Barueri            64.91
Santos             64.87
Corinthians        63.42
Flamengo           63.40
Atletico-MG        62.99
Cruzeiro           61.65
Paysandu           60.20
Fluminense         59.17
Bragantino         57.89
Sao Caetano        57.75
Guarani            57.44
Goias              56.27
Coritiba           55.71
Sport              55.66
Parana             55.65
Portuguesa         55.56
Botafogo-RJ        55.30
Fortaleza          54.85
Criciuma           54.76
Vasco              53.93
Figueirense        53.88
Juventude          53.85
Ponte Preta        53.22
Vitoria            52.86
Ipatinga           52.63
Bahia              51.33
Ceara              51.13
Nautico            50.88
Santo Andre        50.88
America-MG         47.95
Chapecoense        45.71
Cuiaba             45.03
Avai               44.36
Joinville          43.86
Atletico-GO        4

In [68]:
# Selecionando times com melhores aproveitamentos como visitante
dados_aproveitamento_fora = dados_times_full['aproveitamento fora de casa']
top_aproveitamentos = dados_aproveitamento_fora.sort_values(ascending=False)
top_aproveitamentos.head(10)

time
Palmeiras        41.96
Sao Paulo        41.65
Flamengo         41.22
Cruzeiro         40.34
Corinthians      39.74
Atletico-MG      37.96
Internacional    37.52
Fluminense       36.92
Bragantino       36.40
Cuiaba           36.26
Name: aproveitamento fora de casa, dtype: float64

In [69]:
# Definindo função para verificar melhora de aproveitamento
def melhora_aproveitamento(casa, fora):
    return casa - fora

# Verificando times com melhoras mais significativas ao jogar em casa (em pontos percentuais)
dados_aproveitamento = dados_times_full[['jogos', 'aproveitamento', 'aproveitamento em casa', 'aproveitamento fora de casa']]
dados_aproveitamento.insert(4, 'melhora ao jogar em casa (pp)', melhora_aproveitamento(dados_aproveitamento['aproveitamento em casa'], dados_aproveitamento['aproveitamento fora de casa']))         
maiores_melhoras = dados_aproveitamento.sort_values(by='melhora ao jogar em casa (pp)', ascending=False)

maiores_melhoras.head(10)

,jogos,aproveitamento,aproveitamento em casa,aproveitamento fora de casa,melhora ao jogar em casa (pp)
time,,,,,
Barueri,38,42.98,64.91,21.05,43.86
Ipatinga,38,30.70,52.63,8.77,43.86
Paysandu,134,38.31,60.20,16.42,43.78
Guarani,130,37.69,57.44,17.95,39.49
Athletico-PR,780,47.69,65.38,30.00,35.38
Criciuma,168,37.30,54.76,19.84,34.92
Portuguesa,114,38.30,55.56,21.05,34.51
Joinville,38,27.19,43.86,10.53,33.33
Nautico,190,35.09,50.88,19.30,31.58


In [70]:
# Selecionando times que menos sentem a diferença ao jogar como visitante
menores_melhoras = dados_aproveitamento.sort_values(by='melhora ao jogar em casa (pp)')
menores_melhoras.head(10)

,jogos,aproveitamento,aproveitamento em casa,aproveitamento fora de casa,melhora ao jogar em casa (pp)
time,,,,,
America-RN,38,14.91,15.79,14.04,1.75
Cuiaba,114,40.64,45.03,36.26,8.77
Atletico-GO,266,36.97,42.61,31.33,11.28
Brasiliense,42,33.33,39.68,26.98,12.70
Gremio Prudente,38,27.19,35.09,19.30,15.79
Chapecoense,265,36.10,45.71,26.57,19.14
Avai,266,33.96,44.36,23.56,20.80
Cruzeiro,704,50.99,61.65,40.34,21.31
Botafogo-RJ,696,44.59,55.30,33.81,21.49


In [91]:
# Observamos que os times que sofrem os menores aumentos em pontos percentuais possuem aproveitamento baixo tanto em casa como fora
# Dessa forma, vamos fazer a análise observando times que possuem apenas aproveitamento superior a 50% nos jogos como mandante
selecao = dados_aproveitamento['aproveitamento em casa'] > 50
menores_melhoras = dados_aproveitamento[selecao].sort_values(by='melhora ao jogar em casa (pp)')

menores_melhoras.head(10)

,jogos,aproveitamento,aproveitamento em casa,aproveitamento fora de casa,melhora ao jogar em casa (pp)
time,,,,,
Cruzeiro,704,50.99,61.65,40.34,21.31
Botafogo-RJ,696,44.59,55.30,33.81,21.49
Bragantino,152,47.15,57.89,36.40,21.49
Vasco,628,42.99,53.93,32.06,21.87
Flamengo,818,52.28,63.40,41.22,22.18
Fluminense,818,48.04,59.17,36.92,22.25
Palmeiras,734,53.45,64.94,41.96,22.98
Corinthians,780,51.58,63.42,39.74,23.68
Sao Caetano,172,45.74,57.75,33.72,24.03


In [92]:
# Criando dataframe para armazenar dados dos vencedores
dados_vencedores = pd.DataFrame()
dados_vencedores['time'] = times
dados_vencedores.set_index('time', inplace=True)

# Verificando quantos gols cada time marcou nas partidas que venceu em casa e fora
gols_casa = vitorias_casa[['mandante','mandante_Placar']]
gols_fora = vitorias_fora[['visitante','visitante_Placar']]

# Lembrando que o Ipatinga nunca venceu fora de casa, portanto não marcou nenhum gol em vitórias fora de casa
ipatinga = pd.DataFrame({'visitante': ['Ipatinga'], 'visitante_Placar': [0]})
gols_fora = pd.concat([gols_fora, ipatinga], ignore_index=True)

# Agora sim, verificando quantos gols cada time marcou nas partidas que venceu em casa e fora
gols_casa = gols_casa.groupby('mandante').sum().sort_index() 
gols_fora = gols_fora.groupby('visitante').sum().sort_index()

# Inserindo dados ao dataframe
dados_vencedores.insert(0, 'gols pró', gols_casa.values + gols_fora.values)
dados_vencedores.head()

,gols pró
time,
America-MG,117
America-RN,9
Athletico-PR,700
Atletico-GO,160
Atletico-MG,757


In [93]:
# Inserindo dados de número de vitórias
dados_vencedores.insert(0, 'vitórias', dados_times_full['vitórias'])

In [94]:
# Definindo função para verificar quantos gols por jogo
def gols_por_jogo(gols, jogos):
    return round(gols / jogos, 2)

# Inserindo quantos gols foram feito por vitória
dados_vencedores.insert(2, 'gols/vitória', gols_por_jogo(dados_vencedores['gols pró'], dados_vencedores['vitórias']))

In [95]:
# Selecionando times que mais marcam gols ao vencer
dados_vencedores.sort_values(by='gols/vitória', ascending=False, inplace=True)
dados_vencedores.head(15)

,vitórias,gols pró,gols/vitória
time,,,
Barueri,12,35,2.92
Gremio Prudente,7,20,2.86
Paysandu,41,105,2.56
Santa Cruz,15,38,2.53
Portuguesa,31,76,2.45
Nautico,54,130,2.41
Vitoria,123,297,2.41
Parana,83,199,2.40
Santos,340,813,2.39


In [102]:
# Selecionando times que mais marcam gols ao vencer (mínimo de 50 vitórias)
selecao = dados_vencedores['vitórias'] > 50
dados_vencedores_minimo50vitorias = dados_vencedores[selecao]
dados_vencedores_minimo50vitorias = dados_vencedores_minimo50vitorias.sort_values(by='gols/vitória', ascending=False)
dados_vencedores_minimo50vitorias.head(15)

,vitórias,gols pró,gols/vitória
time,,,
Nautico,54,130,2.41
Vitoria,123,297,2.41
Parana,83,199,2.40
Santos,340,813,2.39
Goias,209,496,2.37
Palmeiras,329,775,2.36
Cruzeiro,304,714,2.35
Atletico-MG,327,757,2.31
Sao Paulo,365,832,2.28


In [103]:
# Selecionando times que menos marcam gols ao vencer
dados_vencedores.sort_values(by='gols/vitória', inplace=True)
dados_vencedores.head(15)

,vitórias,gols pró,gols/vitória
time,,,
CSA,8,12,1.50
Cuiaba,34,61,1.79
Chapecoense,70,137,1.96
Juventude,85,169,1.99
America-MG,58,117,2.02
Ponte Preta,114,236,2.07
Ceara,72,149,2.07
Sport,131,271,2.07
Guarani,36,75,2.08


In [104]:
# Criando dataframe para armazenar dados dos perdedores
dados_perdedores = pd.DataFrame()
dados_perdedores['time'] = times
dados_perdedores.set_index('time', inplace=True)

# Verificando quantos gols cada time sofreu nas partidas que perdeu em casa e fora
gols_casa = derrotas_casa[['mandante','visitante_Placar']]
gols_fora = derrotas_fora[['visitante','mandante_Placar']]
gols_casa = gols_casa.groupby('mandante').sum().sort_index() 
gols_fora = gols_fora.groupby('visitante').sum().sort_index()

# Inserindo dados ao dataframe
dados_perdedores.insert(0, 'gols sofridos', gols_casa.values + gols_fora.values)

# Inserindo dados de derrotas
dados_perdedores.insert(0, 'derrotas', dados_times_full['derrotas'])

# Inserindo quantos gols foram sofridos por derrota
dados_perdedores.insert(2, 'gols sofridos/derrota', gols_por_jogo(dados_perdedores['gols sofridos'], dados_perdedores['derrotas']))

In [105]:
# Selecionando times que mais sofrem gols ao perder
dados_perdedores.sort_values(by='gols sofridos/derrota', ascending=False, inplace=True)
dados_perdedores.head(15)

,derrotas,gols sofridos,gols sofridos/derrota
time,,,
Paysandu,62,170,2.74
Ipatinga,21,55,2.62
Nautico,98,253,2.58
Juventude,125,317,2.54
Vasco,234,587,2.51
Santa Cruz,47,118,2.51
Criciuma,80,198,2.48
Ponte Preta,157,384,2.45
Portuguesa,45,109,2.42


In [106]:
# Selecionando times que menos sofrem gols ao perder
dados_perdedores.sort_values(by='gols sofridos/derrota', inplace=True)
dados_perdedores.head(15)

,derrotas,gols sofridos,gols sofridos/derrota
time,,,
Cuiaba,43,76,1.77
Joinville,21,41,1.95
Sao Caetano,66,132,2.00
Corinthians,225,453,2.01
Ceara,102,207,2.03
Botafogo-RJ,259,537,2.07
Sao Paulo,224,469,2.09
Internacional,239,499,2.09
Cruzeiro,235,494,2.10


In [114]:
# Verificando, no geral, quais times fazem mais gols no campeonato
dados_gols_feitos = dados_times_full[['jogos', 'gols pró']]
dados_gols_feitos.insert(2, 'gols/jogo', gols_por_jogo(dados_gols_feitos['gols pró'], dados_gols_feitos['jogos']))
    
mais_gols_jogo = dados_gols_feitos.sort_values(by='gols/jogo', ascending=False)
mais_gols_jogo.head(15)

,jogos,gols pró,gols/jogo
time,,,
Barueri,38,59,1.55
Palmeiras,734,1077,1.47
Atletico-MG,779,1131,1.45
Cruzeiro,704,1013,1.44
Santos,818,1178,1.44
Paysandu,134,193,1.44
Flamengo,818,1169,1.43
Sao Paulo,818,1163,1.42
Gremio,738,1019,1.38


In [118]:
# Verificando, no geral, quais times sofrem mais gols no campeonato
dados_gols_sofridos = dados_times_full[['jogos', 'gols sofridos']]
dados_gols_sofridos.insert(2, 'gols/jogo', gols_por_jogo(dados_gols_sofridos['gols sofridos'], dados_gols_sofridos['jogos']))
    
mais_gols_jogo = dados_gols_sofridos.sort_values(by='gols/jogo', ascending=False)
mais_gols_jogo.head(15)

,jogos,gols sofridos,gols/jogo
time,,,
America-RN,38,80,2.11
Santa Cruz,76,145,1.91
Paysandu,134,245,1.83
Ipatinga,38,67,1.76
Gremio Prudente,38,64,1.68
Nautico,190,318,1.67
Brasiliense,42,68,1.62
Santo Andre,38,61,1.61
Criciuma,168,264,1.57
